Lets take a look at a library called numexpr. Take a look at the [github page](https://github.com/pydata/numexpr) for it. As you can see we can get significant speed improvments using this!

Start by taking a look at my previous post about [Neural Networks in Python using numpy](http://blog.schlerp.net/2016/7/neural-networks-in-python-1-numpy/). 

Let's swap the functions in the file we wrote in the original post to use numexpr where ever we can without changing the structure too much and making it difficult to understand whats happening.

In [ ]:
import numpy as np
import numexpr as ne

class NENonlinear(object):
    """
    Nonlinear
    ---------
    this is used to set up a non linear for a
    network. The idea is you can instantiate it 
    and set what type of non linear function it 
    will be for that particular neaural network
    """
    
    _FUNC_TYPES = ('sigmoid',
                   'softmax',
                   'relu',
                   'tanh',
                   'softplus')
    
    def __init__(self, func_type='sigmoid'):
        if func_type in self._FUNC_TYPES:
            if func_type == self._FUNC_TYPES[0]:
                # sigmoid
                self._FUNCTION = self._FUNC_TYPES[0]
            elif func_type == self._FUNC_TYPES[1]:
                # softmax
                self._FUNCTION = self._FUNC_TYPES[1]
            elif func_type == self._FUNC_TYPES[2]:
                # relu
                self._FUNCTION = self._FUNC_TYPES[2]
            elif func_type == self._FUNC_TYPES[3]:
                # tanh
                self._FUNCTION = self._FUNC_TYPES[3]
            elif func_type == self._FUNC_TYPES[4]:
                # tanh
                self._FUNCTION = self._FUNC_TYPES[4]
        else:
            # default to sigmoid on invalid choice?
            print("incorrect option `{}`".format(func_type))
            print("defaulting to sigmoid")
            self._init_sigmoid()
    
    def __call__(self, x, derivative=False):
        ret = None
        if self._FUNCTION == self._FUNC_TYPES[0]:
            # sigmoid
            if derivative:
                ret = ne.evaluate('x*(1-x)')
            else:
                try:
                    ret = ne.evaluate('1/(1+exp(-x))')
                except RuntimeWarning:
                    ret = 0.0
        elif self._FUNCTION == self._FUNC_TYPES[1]:
            # softmax
            if derivative:
                # from below + http://www.derivative-calculator.net/
                ret = ne.evaluate('2*(exp(x)/sum(exp(x)))')
            else:
                # from: https://gist.github.com/stober/1946926
                #e_x = np.exp(x - np.max(x))
                #ret = e_x / e_x.sum()
                # from: http://peterroelants.github.io/posts/neural_network_implementation_intermezzo02/
                ret = ne.evaluate('exp(x)/sum(exp(x), axis=0)')
        elif self._FUNCTION == self._FUNC_TYPES[2]:
            # relu
            if derivative:
                # from below + http://www.derivative-calculator.net/
                ret = ne.evaluate('2*(abs(x))')
            else:
                ret = ne.evaluate('x*(abs(x))')
        elif self._FUNCTION == self._FUNC_TYPES[3]:
            # tanh
            if derivative:
                # from my own memory of calculus :P
                ret = ne.evaluate('1.0-x**2')
            else:
                ret = ne.evaluate('tanh(x)')
        elif self._FUNCTION == self._FUNC_TYPES[3]:
            # softmax
            if derivative:
                # from wikipedia
                ret = ne.evaluate('1.0/(1+exp(-x))')
            else:
                ret = ne.evaluate('log(1+exp(x))')
        return ret

See how we have kept the structure very similar? numexpr is an exceptionally easy modification to add to most nonlinears!

Now lets implement an N-layered Neural network with numexpr and numpy!

In [ ]:
class NENNN(object):
    """N-layered neural network"""
    def __init__(self, inputs, weights, outputs, alpha):
        self.trained_loops = 0
        self.inputs = inputs
        self.outputs = outputs
        self._ALPHA = alpha
        self._num_of_weights = len(weights)
        self._LAYER_DEFS = {}
        self.WEIGHT_DATA = {}
        self.LAYER_FUNC = {}
        self.LAYERS = {}
        for i in range(self._num_of_weights):
            #(in, out, nonlin)
            self._LAYER_DEFS[i] = {'in': weights[i][0],
                                   'out': weights[i][1],
                                   'nonlin': weights[i][2]}
        print(self._LAYER_DEFS)
        self._init_layers()
    
    def _init_layers(self):
        for i in range(self._num_of_weights):
            _in = self._LAYER_DEFS[i]['in']
            _out = self._LAYER_DEFS[i]['out']
            _nonlin = self._LAYER_DEFS[i]['nonlin']
            self.WEIGHT_DATA[i] = np.random.randn(_in, _out)
            self.LAYER_FUNC[i] = _nonlin
    
    def reset(self):
        self._init_layers()
    
    def _do_layer(self, prev_layer, next_layer, nonlin):
        """Does the actual calcs between layers :)"""
        ret = nonlin(np.dot(prev_layer, next_layer))
        return ret

    def train(self, x, y, train_loops=100):
        for j in range(train_loops):
            # set up layers
            prev_layer = x
            prev_y = y
            next_weight = None
            l = 0
            self.LAYERS[l] = x
            for i in range(self._num_of_weights):
                l += 1
                next_weight = self.WEIGHT_DATA[i]
                nonlin = self.LAYER_FUNC[i]
                current_layer = self._do_layer(prev_layer, next_weight, nonlin)
                self.LAYERS[l] = current_layer
                prev_layer = current_layer
            last_layer = current_layer
            #print(last_layer)
            #
            #layer2_error = y - layer2
            #layer2_delta = layer2_error * self.non_lin(layer2, derivative=True)
            
            #layer1_error = layer2_delta.dot(self.w_out.T)
            #layer1_delta = layer1_error * self.non_lin(layer1, derivative=True)
            
            #self.w_out += self._ALPHA * layer1.T.dot(layer2_delta)
            #self.w_in += self._ALPHA * layer0.T.dot(layer1_delta)              
            
            # calculate errors
            output_error = ne.evaluate('y - last_layer')
            output_nonlin = self.LAYER_FUNC[self._num_of_weights - 1]
            output_delta = output_error * output_nonlin(last_layer, derivative=True)

            prev_delta = output_delta
            prev_layer = last_layer
            for i in reversed(range(self._num_of_weights)):
                weight = self.WEIGHT_DATA[i]
                current_weight_error = prev_delta.dot(weight.T)
                current_weight_nonlin = self.LAYER_FUNC[i]
                current_weight_delta = current_weight_error * current_weight_nonlin(self.LAYERS[i], derivative=True)
                # backpropagate error
                self.WEIGHT_DATA[i] += self._ALPHA * self.LAYERS[i].T.dot(prev_delta)
                prev_delta = current_weight_delta
                
            # increment the train counter, so i can see how many 
            # loops my pickled nets have trained
            self.trained_loops += 1
            
            # output important info
            if (j % (train_loops/10)) == 0:
                print("loop: {}".format(j))
                #print("Layer1 Error: {}".format(np.mean(np.abs(layer1_error))))                
                #print("Layer2 Error: {}".format(np.mean(np.abs(layer2_error))))
                print("Guess: ")
                print(last_layer[0])
                #print("output delta: ")
                #print(np.round(output_delta, 2))
                print("Guess (rounded): ")
                print(np.round(last_layer[0], 1))
                print("Actual: ")
                print(y[0])
        
    def guess(self, x):
        prev_layer = x
        prev_y = y
        next_weight = None
        l = 0
        self.LAYERS[l] = x
        for i in range(self._num_of_weights):
            l += 1
            next_weight = self.WEIGHT_DATA[i]
            nonlin = self.LAYER_FUNC[i]
            current_layer = self._do_layer(prev_layer, next_weight, nonlin)
            self.LAYERS[l] = current_layer
            prev_layer = current_layer
        last_layer = current_layer
        return last_layer

Enjoy!